# Previsão de Saída e Permanência de Clientes

Nesta análise iremos prever se os clientes do Banco Beta vão permanecer ou cancelar seus serviços com base em vários fatores, incluindo sua pontuação de crédito, localização geográfica, gênero, idade, tempo de relacionamento com o banco, saldo em conta, número de produtos utilizados, posse de cartão de crédito, atividade da conta e salário estimado. Para isso, usaremos um conjunto de dados que contém informações relevantes sobre os clientes.

## Sumário

1. [Iniciação](#inic)

2. [Pré-processamento de Dados](#ppd)

    A. [Excluindo Colunas](#ec)

    B. [Valores Ausentes](#va)

    C. [Colunas Categóricas](#cc)

    D. [Colunas Numéricas](#cn)

    E. [Features, Target e Divisão de Dados](#ftdd)

3. [Modelo sem Balanceamento de Classe](#msbc)

    A. [Árvore de Decisão](#ad)

    B. [Floresta Aleatória](#fa)

    C. [Regressão Logística](#rl)

4. [Balanceamento de Classes](#bc)

    A. [Ajuste de Classe Ponderada](#acp)

    B. [Superamostragem](#sa)

5. [Teste Final](#tf)

6. [Conclusão](#co)

## Iniciação <a id="inic"></a>

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [2]:
try:
    df = pd.read_csv('Churn.csv')
except:
    df = pd.read_csv('/datasets/Churn.csv')

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Utilizaremos a coluna `Exited` como `target` para prever a saída de clientes, enquanto as demais colunas do conjunto de dados serão utilizadas como `features` para a criação do modelo. Para preparar os dados para a análise, primeiro é necessário preencher os valores ausentes na coluna `Tenure`. Em seguida, para as colunas categóricas, devemos criar variáveis dummy. Por fim, padronizar as colunas numéricas, exceto as que contêm valores binários, como `HasCrCard` e `IsActiveMember`.

## Pré-processamento de Dados <a id="ppd"></a>

### Excluindo Colunas <a id="ec"></a>

Existem três colunas no conjunto de dados que não são relevantes para a análise: `RowNumber`, `CustomerId` e `Surname`. A coluna `RowNumber` é apenas uma numeração sequencial para cada linha do conjunto de dados, enquanto `CustomerId` e `Surname` são identificadores únicos para cada cliente. Essas informações não são úteis para o treinamento do modelo, portanto, podemos descartá-las para simplificar a análise.

In [4]:
data = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           9091 non-null   float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


### Valores Ausentes <a id="va"></a>

In [5]:
data['Tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

Após notarmos os valores ausentes iremos preenchê-los com o valor da mediana.

In [6]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median())
data['Tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0.])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


Valores preenchidos.

### Colunas Categóricas <a id="cc"></a>

Analisaremos as 2 colunas categóricas `Geography` e `Gender` e seus valores.

In [8]:
data['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [9]:
data['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

Observados os valores das colunas categóricas `Geography` e `Gender` temos em `Geography` 3 valores:
`Spain`, `Germany` e `France`. Ao criar as variáveis dummy para essa coluna, ela será substituída por 3 colunas: `Geography_Spain`, `Geography_Germany` e `Geography_France`. Cada coluna terá o valor `1` na observação em que a coluna `Geography` tiver o país correspondente como valor e, caso contrário, o valor será `0`. O mesmo será feito para a coluna `Gender`. Para criar as variáveis dummy, será usado a função `pd.get_dummies` em toda a tabela `data`, já que essas são as únicas colunas categóricas. Podemos descartar uma das colunas dummy para ambos os cenários, porque um valor `0` para `Spain` e `Germany`, por exemplo, implica diretamente um valor `1` para `France`. Isso pode ser feito configurando o parâmetro `drop_first=True`.

In [10]:
data = pd.get_dummies(data, drop_first=True)
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


### Colunas Numéricas <a id="cn"></a>

As colunas numéricas em questão são `CreditScore`, `Age`, `Tenure`, `Balance`, `NumOfProducts` e `EstimatedSalary`. Como essas variáveis não possuem uma escala definida, precisamos escaloná-las para que tenham a mesma importância no modelo. Para fazer isso, usaremos a função `StandardScaler()`, que ajustará as colunas numéricas através do método `fit()` e as transformará em desvios padrão. Dessa forma, é evitado que variáveis com alta dispersão tenham mais peso no modelo do que as outras. Só então, teremos os valores escalonados e prontos para o treinamento do modelo.

In [11]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(data[numeric])
data[numeric] = scaler.transform(data[numeric])
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,-0.326221,0.293517,-1.086246,-1.225848,-0.911583,1,1,0.021886,1,0,0,0
1,-0.440036,0.198164,-1.448581,0.117350,-0.911583,0,1,0.216534,0,0,1,0
2,-1.536794,0.293517,1.087768,1.333053,2.527057,1,0,0.240687,1,0,0,0
3,0.501521,0.007457,-1.448581,-1.225848,0.807737,0,0,-0.108918,0,0,0,0
4,2.063884,0.388871,-1.086246,0.785728,-0.911583,1,1,-0.365276,0,0,1,0


### Features, Target e Divisão dos Dados <a id="ftdd"></a>

Definiremos a variável objetivo (target) que será a coluna `Exited`, enquanto as outras colunas serão as características (features). Para avaliar o desempenho do modelo, dividiremos o conjunto de dados em conjuntos de treinamento, validação e teste, com proporções de `60%`, `20%` e `20%`, respectivamente. Para tal, usaremos a função `train_test_split()` duas vezes. Na primeira vez, dividiremos o conjunto de dados em um conjunto de treinamento e um segundo conjunto definindo o parâmetro `test_size=0.4` (que representa a porcentagem do conjunto de dados que será o segundo conjunto). Na segunda vez, dividiremos o segundo conjunto em dois conjuntos iguais (`test_size=0.5`) para obter os conjuntos de validação (20% do conjunto de dados original) e teste (20% do conjunto de dados original). O `random_state` será definido como `12345` e que será mantido no treinamento do modelo.

In [12]:
features = data.drop('Exited', axis=1)
target = data['Exited']
features_train, features_test_valid, target_train, target_test_valid = train_test_split(features, target,\
test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_test_valid, \
target_test_valid, test_size=0.5, random_state=12345)

print(len(features_train), len(target_train), len(features_valid), len(target_valid), len(features_test), \
len(target_test))

6000 6000 2000 2000 2000 2000


## Modelo sem Balanceamento de Classe <a id="msbc"></a>

### Árvore de Decisão <a id="ad"></a>

Utilizaremos a função `DecisiontreeClassifier()`. Vamos definir dois hiperparâmetros: `random_state` e `max_depth`. O `random_state` deve ser o mesmo em todos os casos, portanto, vamos atribuir um valor fixo (12345). O `max_depth` é o hiperparâmetro que vamos ajustar. Vamos percorrer uma série de valores para o `max_depth` (de 1 a 10) e obter a pontuação `f1` e `AUC-ROC`, ambas métricas de qualidade do modelo. O `f1_score` processa o alvo do conjunto de validação e as previsões. A função `roc_auc_score` processa o alvo do conjunto de validação com as probabilidades da classe positiva de cada observação no conjunto de validação. Para isso, vamos utilizar a função `predict_proba()`.

In [13]:
for i in range(1, 11):  
    dt_model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    dt_model.fit(features_train, target_train)
    dt_pred_valid = dt_model.predict(features_valid)
    
    probabilities_valid = dt_model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    
    print('Max depth', i, 'F1 score =', f1_score(target_valid, dt_pred_valid), 'AUC-ROC score =', \
         roc_auc_score(target_valid, probabilities_one_valid))

Max depth 1 F1 score = 0.0 AUC-ROC score = 0.6925565119556736
Max depth 2 F1 score = 0.5217391304347825 AUC-ROC score = 0.7501814673449512
Max depth 3 F1 score = 0.4234875444839857 AUC-ROC score = 0.7973440741838507
Max depth 4 F1 score = 0.5528700906344411 AUC-ROC score = 0.813428129858032
Max depth 5 F1 score = 0.5406249999999999 AUC-ROC score = 0.8221680508592478
Max depth 6 F1 score = 0.5696969696969697 AUC-ROC score = 0.8164631712023421
Max depth 7 F1 score = 0.5320813771517998 AUC-ROC score = 0.8138530658907929
Max depth 8 F1 score = 0.5454545454545454 AUC-ROC score = 0.8119854644656693
Max depth 9 F1 score = 0.5633802816901409 AUC-ROC score = 0.7801515554775917
Max depth 10 F1 score = 0.5406162464985994 AUC-ROC score = 0.7658451236699957


Observamos que o melhor `f1_score` (~0,57) é obtido no `max_depth` 6, com um valor `AUC-ROC` de aproximadamente 0,82.

### Floresta Aleatória <a id="fa"></a>

Para esse modelo usaremos a função `RandomForestClassifier()`. Mudaremos os hiperparâmetros `max_depth` e `n_estimatorsSerá`. Para o mesmo, criaremos uma lista vazia e, em seguida, um loop que percorra valores de `max_depth` e, dentro desse loop, outro loop que percorra valores de `n_estimators`. Usaremos esse loop para criar modelos com diferentes combinações de valores de `max_depth` e `n_estimators` que serão armazenados na lista. A partir dessa lista, escolheremos o modelo com a pontuação `f1` mais alta.

In [14]:
rf = []
for i in range(1, 11):
    for j in range(10, 101, 10):
        rf_model = RandomForestClassifier(random_state=12345, max_depth=i, n_estimators=j)
        rf_model.fit(features_train, target_train)
        rf.append(rf_model)
    
print(max(rf, key=lambda rf_model: f1_score(rf_model.predict(features_valid), target_valid)))

RandomForestClassifier(max_depth=10, n_estimators=10, random_state=12345)


O modelo `Floresta Aleatória` com o maior `f1_score` tem os hiperparâmetros `max_depth=10` e `n_estimators=10`. Portanto, iremos treiná-lo especificamente com esses hiperparâmetros e obter um `f1_score` e um `roc_auc_score`.

In [15]:
best_rf_model = RandomForestClassifier(random_state=12345, max_depth=10, n_estimators=10)
best_rf_model.fit(features_train, target_train)
best_rf_pred = best_rf_model.predict(features_valid)

probabilities_rf_valid=best_rf_model.predict_proba(features_valid)
probabilities_rf_one_valid=probabilities_rf_valid[:, 1]

print('F1 score =', f1_score(target_valid, best_rf_pred), 'AUC-ROC =', \
      roc_auc_score(target_valid, probabilities_rf_one_valid))

F1 score = 0.5869894099848714 AUC-ROC = 0.8461436676969979


A pontuação `f1` é `~0.59`, com uma pontuação `AUC-ROC` de `~0.85`

### Regressão Logística <a id="rl"></a>

Neste ultilizaremos a função `LogisticRegression()`. O `random_state` é o mesmo, mas os hiperparâmetros `max_depth` e `n_estimators` não se aplicam, portanto, só é necessário definir um solucionador (solver) que será o `liblinear`.

In [16]:
lr_model = LogisticRegression(random_state=12345, solver='liblinear')
lr_model.fit(features_train, target_train)
lr_valid_pred = lr_model.predict(features_valid)

probabilities_lr_valid = lr_model.predict_proba(features_valid)
probabilities_lr_one_valid = probabilities_lr_valid[:, 1]

print('F1 score =', f1_score(target_valid, lr_valid_pred), 'AUC-ROC =', \
     roc_auc_score(target_valid, probabilities_lr_one_valid))

F1 score = 0.33108108108108103 AUC-ROC = 0.7587497504824008


- Conclusão

Entre os três modelos testados, o `Random Forest Classifier` com hiperparâmetros `max_depth=10` e `n_estimators=10` obteve o melhor desempenho, com a pontuação `f1` mais alta (cerca de 0,59) e `AUC-ROC` (cerca de 0,84). Portanto, este será o modelo usado daqui pra frente.

## Balanceamento de Classes <a id="bc"></a>

Agora analisaremos o desequilíbrio de classe para determinar as proporções de cada classe no alvo do conjunto de treinamento. Para o mesmo, utilizaremos a função `value_counts()` com o parâmetro `normalize=True`.

In [17]:
target_train.value_counts(normalize=True)

0    0.800667
1    0.199333
Name: Exited, dtype: float64

No conjunto de treinamento, a classe negativa (0) representa cerca de `80%` dos dados, enquanto a classe positiva (1) representa cerca de `20%`. Isso significa que há `quatro vezes` mais instâncias da classe `0` do que da classe `1`. Para abordar esse desequilíbrio de classe, veremos duas abordagens diferentes.

### Ajuste de Classe Ponderada <a id="acp"></a>

Para utilizar essa abordagem, basta definir o hiperparâmetro `class_weight='balanced'` durante o treinamento do modelo. Isso fará com que a classe mais rara (1 neste caso) tenha mais peso.

In [18]:
bal_rf_model = RandomForestClassifier(random_state=12345, max_depth=10, n_estimators=10, \
                                      class_weight='balanced')
bal_rf_model.fit(features_train, target_train)
bal_rf_pred = bal_rf_model.predict(features_valid)

proba_bal_rf_valid = bal_rf_model.predict_proba(features_valid)
proba_bal_rf_one_valid = proba_bal_rf_valid[:, 1]

print('F1 score =', f1_score(target_valid, bal_rf_pred), 'AUC-ROC =', \
      roc_auc_score(target_valid, proba_bal_rf_one_valid))

F1 score = 0.6038647342995168 AUC-ROC = 0.8418065981526625


O valor de `f1 score` já está melhor do que antes, mas o valor `AUC-ROC` teve uma queda quase imperceptível. A taxa de verdadeiros positivos certamente diminuiu um pouco.

### Superamostragem <a id="sa"></a>

Nesse método, iremos basicamente repetir a classe minoritária e suas observações várias vezes até que ela esteja equilibrada em relação à outra classe. Constatamos anteriormente que existem `4 vezes` mais valores `0` do que `1`, então iremos repetir os valores `1` e suas observações `4 vezes` para igualar o número de zeros no conjunto de treinamento. Depois de fazer isso, precisaremos embaralhá-los usando a função `shuffle()` para que a aprendizagem não se torne muito fácil.

In [19]:
def upsample(features, target, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

print(features_upsampled.shape, target_upsampled.shape)

(9588, 11) (9588,)


Agora podemos treinar o modelo usando as `features` e `targets` superamostradas.

In [20]:
ups_rf_model = RandomForestClassifier(random_state=12345, max_depth=10, n_estimators=10)
ups_rf_model.fit(features_upsampled, target_upsampled)
ups_rf_pred = ups_rf_model.predict(features_valid)

proba_ups_rf_valid = ups_rf_model.predict_proba(features_valid)
proba_ups_rf_one_valid = proba_ups_rf_valid[:, 1]

print('F1 score =', f1_score(target_valid, ups_rf_pred), 'AUC-ROC =', \
      roc_auc_score(target_valid, proba_ups_rf_one_valid))

F1 score = 0.5836909871244635 AUC-ROC = 0.8335694929197491


O `f1 score` é menor do que o que obtivemos ao usar o ajuste de peso de classe. Notamos também que `AUC-ROC` diminuiu, mas com uma pequena diferença. A taxa de verdadeiros positivos deve ter diminuido.

- Conclusão

Prosseguiremos com a abordagem de ajuste de peso de classe, já que ela apresenta a pontuação de `f1` mais alta, que é de `0,60`.

## Teste Final <a id="tf"></a>

Agora utilizaremos o modelo (com o ajuste dos pesos das classes) no conjunto de teste. Antes disso, é necessário treinar o modelo utilizando tanto o conjunto de treinamento quanto o de validação, e para isso, iremos combiná-los usando a função `pd.concat()`.

In [21]:
features_train_final = pd.concat([features_train] + [features_valid])
target_train_final = pd.concat([target_train] + [target_valid])

final_rf_model = RandomForestClassifier(random_state=12345, max_depth=10, n_estimators=10, \
                                        class_weight='balanced')
final_rf_model.fit(features_train_final, target_train_final)
final_rf_pred = final_rf_model.predict(features_test)

proba_rf_test = final_rf_model.predict_proba(features_test)
proba_rf_one_test = proba_rf_test[:, 1]

print('F1 score =', f1_score(target_test, final_rf_pred), 'AUC-ROC =', \
      roc_auc_score(target_test, proba_rf_one_test))

F1 score = 0.6187214611872146 AUC-ROC = 0.8509656393397403


A pontuação final de `f1` é `~0,62` que é maior que o limite mínimo de `0,59`.

## Conclusão <a id="co"></a>

Após o processamento do conjunto de dados, que incluiu escalar colunas numéricas, preencher valores ausentes e obter colunas dummy a partir de variáveis categóricas, o conjunto de dados foi dividido em treino e teste. Então, treinamos três modelos de classificação diferentes, `Decision Tree`, `Random Forest` e `Logistic Regression Classifier`, sem considerar o desequilíbrio de classes, que apresentou uma proporção de `4:1`, com a classe negativa representando cerca de `80%` dos dados e a classe positiva cerca de `20%`.

Após avaliar o desempenho dos modelos, foi constatado que o `Random Forest` apresentou a melhor métrica, com um alto valor de `f1-score` de aproximadamente `0,59` e um valor de AUC-ROC de `~0,85`. Em seguida, consideramos o desequilíbrio de classes e testamos duas abordagens para solucionar o problema: ajuste de peso de classe e superamostragem. Optamos pela primeira abordagem, que resultou em um `f1-score` mais alto de `0,59`, mesmo que a outra opção tivesse um valor `AUC-ROC` mais alto.

Treinamos o modelo com os dados de treinamento e validação e o aplicamos ao conjunto de teste, obtendo um `f1-score` de `~0,62`.